In [19]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from imutils import perspective
from imutils import contours
import imutils
import os

In [30]:
disparityMapScale = 35
imageScale = 25
disparityMap = []
isTrain = False
average = 0
sensorHeight = 33.3 #hard to find, so calculated beforehand using known height, known distance, and pixel.
cameraFocalLength = 28
distance = 0
points = []

In [25]:
testLeftImage = cv2.imread('images/test/image1/left.jpg')
testRightImage = cv2.imread('images/test/image1/right.jpg')
image = []

In [4]:
def calibrate():
    if os.path.isfile('./camera/mtx.npy') and os.path.isfile('./camera/dist.npy'):
        return np.load('./camera/mtx.npy'), np.load('./camera/dist.npy')
    else:
        imgpoints = []
        objpoints = []
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        objp = np.zeros((1, boardShape[0] * boardShape[1], 3), np.float32)
        objp[0,:,:2] = np.mgrid[0:boardShape[0], 0:boardShape[1]].T.reshape(-1, 2)
        for filename in os.listdir("images/calibrator2"):
            if (filename.find(".jpg") != -1):

                img = cv2.imread(os.path.join("images/calibrator2",filename))
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                ret, corners = cv2.findChessboardCorners(gray, boardShape, 
                                                         cv2.CALIB_CB_ADAPTIVE_THRESH + 
                                                         cv2.CALIB_CB_FAST_CHECK + 
                                                         cv2.CALIB_CB_NORMALIZE_IMAGE)
                if ret == True:
                    print(filename)
                    objpoints.append(objp)
                    corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
                    imgpoints.append(corners)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        return mtx, dist

In [5]:
def resize(img, percentage):
    return cv2.resize(img, (int(img.shape[1] * percentage / 100), 
                                int(img.shape[0] * percentage / 100)),
                               interpolation = cv2.INTER_AREA)

In [6]:
def downSample(image, times = 1):
    for i in range(0,times):
        image = cv2.pyrDown(image)
    return image

In [7]:
def getCameraMatrix(image):
    h,  w = image.shape[:2]
    newCameraMtx, roi = cv2.getOptimalNewCameraMatrix(calibrator[0], calibrator[1], (w,h), 1, (w,h))
    return newCameraMtx, roi

In [8]:
def getDistance(event, x, y, flags, param):
    global disparityMap, isTrain, average, distance
    if event == cv2.EVENT_LBUTTONDBLCLK:
        total = 0
        resizedDisparityMap = resize(disparityMap, disparityMapScale)
        for u in range(-1,2):
            for v in range(-1,2):
                total += resizedDisparityMap[y + u, x + v]
        average = total / 9
        
        if isTrain:
            print(average)
        else:
#             = -6E-13x4 + 6E-09x3 - 1E-05x2 - 0.0392x + 163.54
#             -2E-09x3 + 2E-05x2 - 0.1015x + 199.39
            distance = -0.000000002*average**(3) + 0.00002*average**(2) - 0.1015*average + 199.39
#             distance = -0.0000000000006*average**(4) + 0.000000006*average**(3) - 0.00001*average**(2) - 0.0392*average + 163.54
            distance = np.around(distance*10,decimals=2)
            print(distance)
#             eq = -0.0112x3 + 2.8479x2 - 255.38x + 9641.7
#             return average

In [9]:
calibrator = calibrate()

In [10]:
def getDisparity(imageLeft, imageRight):
    cameraMtx = getCameraMatrix(imageLeft)
    undistortedL = cv2.undistort(imageLeft, calibrator[0], calibrator[1], None, cameraMtx[0])
    undistortedR = cv2.undistort(imageRight, calibrator[0], calibrator[1], None, cameraMtx[0])

    undistortedL = downSample(undistortedL, 1)
    undistortedR = downSample(undistortedR, 1)
    
    win_size = 5 #3
    min_disp = -10
    max_disp = 326 #min_disp * 9
    num_disp = max_disp - min_disp # Needs to be divisible by 16
    #Create Block matching object. 
    stereo = cv2.StereoSGBM_create(
        minDisparity= min_disp,
        numDisparities = num_disp,
        blockSize = win_size,
        uniquenessRatio = 3,
        speckleWindowSize = 100,
        speckleRange = 32,
        disp12MaxDiff = 5,
        P1 = 8*3*win_size**2,#8*3*win_size**2,
        P2 =32*3*win_size**2
    ) #32*3*win_size**2)

#     stereoResult = stereo.compute(undistortedL, undistortedR)
    return stereo.compute(undistortedL, undistortedR)

In [60]:
folderName =  "images/train"
cv2.namedWindow("image") 
cv2.setMouseCallback("image", getDistance)
distanceToAverageRatio = []
for folder in os.listdir(folderName):
    if (folder.isdigit()):
        distFolder = os.path.join(folderName, folder)
        print(distFolder)
        print(os.listdir(distFolder))
        imgL = cv2.imread(os.path.join(distFolder, 'left.jpg'))
        imgR = cv2.imread(os.path.join(distFolder, 'right.jpg'))
        disparityMap = getDisparity(imgL, imgR)
#         disparityMap = ((disparityMap.astype(np.float32)/ 16)-(-10))/316
#         color= cv2.applyColorMap(disparityMap,cv2.COLORMAP_OCEAN) 
        while True:
            cv2.imshow("image", resize(disparityMap, disparityMapScale)) 
            key = cv2.waitKey(1) & 0xFF
  
            if key == ord("c"):
                print(average)
                distanceToAverageRatio.append([folder, average])
                break
cv2.destroyAllWindows()

images/train/61
['.DS_Store', 'left.jpg', 'right.jpg']
748.51
2402.777777777778
images/train/95
['.DS_Store', 'left.jpg', 'right.jpg']
1052.65
1347.4444444444443
images/train/105
['.DS_Store', 'left.jpg', 'right.jpg']
1088.16
1258.6666666666667
images/train/58
['.DS_Store', 'left.jpg', 'right.jpg']
753.88
2371.222222222222
images/train/67
['.DS_Store', 'left.jpg', 'right.jpg']
853.79
1932.3333333333333
images/train/34
['.DS_Store', 'left.jpg', 'right.jpg']
760.62


KeyboardInterrupt: 

In [54]:
distanceToAverageRatio

[['61', 2401.6666666666665],
 ['95', 1378.3333333333333],
 ['105', 1306.4444444444443],
 ['58', 2379.6666666666665],
 ['67', 1935.6666666666667],
 ['34', 3843.222222222222],
 ['73', 1830.5555555555557],
 ['28', 4811.111111111111],
 ['110', 1027.3333333333333],
 ['43', 2952.3333333333335],
 ['31', 4193.666666666667],
 ['100', 1374.5555555555557],
 ['37', 3610.222222222222],
 ['52', 2578.222222222222],
 ['55', 2578.6666666666665],
 ['64', 2124.0],
 ['90', 1487.2222222222222],
 ['46', 2913.4444444444443],
 ['79', 1717.888888888889],
 ['115', 1207.6666666666667],
 ['70', 2097.6666666666665],
 ['85', 1585.111111111111],
 ['76', 1938.6666666666667],
 ['82', 1833.4444444444443],
 ['49', 2809.1111111111113],
 ['40', 3168.222222222222],
 ['25', 3168.222222222222]]

In [55]:
with open('distanceToAverageRatio.txt', 'w') as filehandle:
    for ratio in distanceToAverageRatio:
        for val in ratio:
            filehandle.write('%s '%val)
        filehandle.write('\n')

In [70]:
def drawAreaToMeasure(event, x, y, flags, param): 
    # grab references to the global variables 
    global points, image
  
    # if the left mouse button was clicked, record the starting 
    # (x, y) coordinates and indicate that cropping is being performed 
    if event == cv2.EVENT_LBUTTONDOWN: 
        points = [(x, y)] 
  
    # check to see if the left mouse button was released 
    elif event == cv2.EVENT_LBUTTONUP: 
        # record the ending (x, y) coordinates and indicate that 
        # the cropping operation is finished 
        points.append((x, y)) 
  
        # draw a rectangle around the region of interest 
        cv2.rectangle(image, points[0], points[1], (0, 255, 0), 2)
        dimension = calculateDimension()
#         (tlM, trM) = midpoint(points[1])
        cv2.putText(image, "Height: {:.1f} cm".format(dimension[0]), (50,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)
        cv2.putText(image, "Width: {:.1f} cm".format(dimension[1]), (50,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)
        cv2.imshow("image", image)

In [31]:
def selectAreaToMeasure():
    global image
    image = testLeftImage.copy()
    image = resize(image, imageScale)
    clone = image.copy()
    cv2.namedWindow("image") 
    cv2.setMouseCallback("image", drawAreaToMeasure) 

    while True: 
        cv2.imshow("image", image) 
        key = cv2.waitKey(1) & 0xFF

        # press 'r' to reset the window 
        if key == ord("r"): 
            image = clone.copy() 

        # if the 'c' key is pressed, break from the loop 
        elif key == ord("q"): 
            break

    cv2.destroyAllWindows()  

In [20]:
def calculateDistance():
    global disparityMap, testLeftImage, testRightImage
    cv2.namedWindow("image") 
    cv2.setMouseCallback("image", getDistance)
    
    disparityMap = getDisparity(testLeftImage, testRightImage)
    while True:
        cv2.imshow("image", resize(disparityMap, disparityMapScale)) 
        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break
    cv2.destroyAllWindows()

In [66]:
def run():
    if isTrain:
        print("here")
    else:
        calculateDistance()
        selectAreaToMeasure()
        dimension = calculateDimension()
        

In [71]:
run()

753.8
4032
4032


In [65]:
def calculateDimension():
    global distance, points
    #since it was resized, we need to get the real pixel inthe original image
    heightInPixel = (points[1][1]/(imageScale/100)) - (points[0][1]/(imageScale/100))
    print(testLeftImage.shape[0])
    realHeight = (distance * sensorHeight * heightInPixel) / (cameraFocalLength * testLeftImage.shape[0])
    realWidth = (realHeight * testLeftImage.shape[1])/testLeftImage.shape[0]
    return [np.round(realHeight*0.1, 2), np.round(realWidth*0.1, 2)]  #so it is in cm

In [63]:
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)